In [1]:
import numpy as np
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import Conv2D, MaxPooling2D, Input
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model


2025-11-22 12:04:02.054686: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-22 12:04:02.056848: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-22 12:04:02.489410: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-22 12:04:05.285429: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
# Download data
(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()

# Normalize inputs
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

# One-hot output vectors
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, 10)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, 10)

In [4]:
def define_functional_model():
    inputs = Input(shape=(32, 32, 3), name="input_layer")
    x = Conv2D(32, (3, 3), use_bias=False, activation='relu', padding='same', name='block1_conv1')(inputs)
    x = Conv2D(32, (3, 3), use_bias=False, activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), name='block1_pool')(x)
    x = Dropout(0.2, name='block1_dropout')(x)
    x = Conv2D(64, (3, 3), use_bias=False, activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(64, (3, 3), use_bias=False, activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), name='block2_pool')(x)
    x = Dropout(0.2, name='block2_dropout')(x)
    x = Conv2D(128, (3, 3), use_bias=False, activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(128, (3, 3), use_bias=False, activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), name='block3_pool')(x)
    x = Dropout(0.2, name='block3_dropout')(x)
    x = Flatten(name='flatten')(x)
    x = Dense(128, use_bias=False, activation='relu', kernel_initializer='he_uniform', name='fc1')(x)
    x = Dropout(0.2, name='fc1_dropout')(x)
    outputs = Dense(10, use_bias=False, activation='softmax', name='predictions')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="cifar10_functional_cnn")
    
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = define_functional_model()
model.summary()


2025-11-22 08:49:50.940678: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "cifar10_functional_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 32)     │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 32)     │         9,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_dropout (Dropout)        │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 64)     │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_dropout (Dropout)        │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 128)      │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 128)      │       147,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_dropout (Dropout)        │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 128)            │       262,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1_dropout (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 10)             │         1,280 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 549,984 (2.10 MB)

 Trainable params: 549,984 (2.10 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:

# Train model (use one-hot labels)
history = model.fit(
    train_data, train_labels_onehot,       # training data (one-hot labels)
    batch_size=64,                        # batch size
    epochs=100,                            # Maximum number of epochs
    validation_split=0.1,                  # Percentage of training data used for validation
)

# Test model: get class predictions and evaluate with one-hot labels
predictions_keras = np.argmax(model.predict(test_data, verbose=0), axis=1)
test_loss, test_accuracy = model.evaluate(test_data, test_labels_onehot, verbose=0)
print(f"Test accuracy: {test_accuracy}")
model.save("CIFAR_v19_vgg.keras")

Epoch 1/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 59s 81ms/step - accuracy: 0.1239 - loss: 2.2857 - val_accuracy: 0.1844 - val_loss: 2.2037
Epoch 2/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 61s 87ms/step - accuracy: 0.2400 - loss: 2.0516 - val_accuracy: 0.3180 - val_loss: 1.9322
Epoch 3/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 63s 89ms/step - accuracy: 0.3063 - loss: 1.9091 - val_accuracy: 0.3472 - val_loss: 1.8351
Epoch 4/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 83s 118ms/step - accuracy: 0.3452 - loss: 1.8092 - val_accuracy: 0.3970 - val_loss: 1.6811
Epoch 5/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 128s 182ms/step - accuracy: 0.3822 - loss: 1.7040 - val_accuracy: 0.4372 - val_loss: 1.5963
Epoch 6/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 97s 138ms/step - accuracy: 0.4182 - loss: 1.6067 - val_accuracy: 0.4658 - val_loss: 1.4938
Epoch 7/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 82s 117ms/step - accuracy: 0.4434 - loss: 1.5383 - val_accuracy: 0.4706 - val_loss: 1.4674
Epoch 8/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 63s 90ms/step - accuracy: 0.4704 - lo

In [3]:
import ANNarchy
from ANNarchy.extensions.ann_to_snn_conversion import ANNtoSNNConverter
ANNarchy.clear()
snn_converter = ANNtoSNNConverter(
    input_encoding='IB', 
    hidden_neuron='IaF',
    read_out='spike_count',
)


ANNarchy 4.8 (4.8.2.5) on linux (posix).


In [4]:
net = snn_converter.load_keras_model("CIFAR_v19_vgg.keras", show_info=True)

2025-11-22 12:04:25.625207: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


* Input layer: input_layer, (32, 32, 3)
* InputLayer skipped.
* Conv2D layer: block1_conv1, (32, 32, 32) 
* Conv2D layer: block1_conv2, (32, 32, 32) 
* MaxPooling2D layer: block1_pool, (16, 16, 32) 
* Dropout skipped.
* Conv2D layer: block2_conv1, (16, 16, 64) 
* Conv2D layer: block2_conv2, (16, 16, 64) 
* MaxPooling2D layer: block2_pool, (8, 8, 64) 
* Dropout skipped.
* Conv2D layer: block3_conv1, (8, 8, 128) 
* Conv2D layer: block3_conv2, (8, 8, 128) 
* MaxPooling2D layer: block3_pool, (4, 4, 128) 
* Dropout skipped.
* Flatten skipped.
* Dense layer: fc1, 128 
    weights: (128, 2048)
    mean -0.0014996225945651531, std 0.03499392047524452
    min -0.1535508632659912, max 0.16072383522987366
* Dropout skipped.
* Dense layer: predictions, 10 
    weights: (10, 128)
    mean -0.0015336459036916494, std 0.1952981948852539
    min -0.49505963921546936, max 0.41484078764915466



In [5]:
predictions_snn = snn_converter.predict(test_data[:300], duration_per_sample=400)

  0%|          | 0/300 [00:00<?, ?it/s]

100%|██████████| 300/300 [1:21:32<00:00, 16.31s/it]


In [6]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

print(classification_report(test_labels[:300], predictions_snn))
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

              precision    recall  f1-score   support

           0       0.82      0.78      0.80        36
           1       0.92      1.00      0.96        24
           2       0.79      0.70      0.75        27
           3       0.58      0.86      0.69        29
           4       0.86      0.78      0.82        23
           5       0.77      0.71      0.74        28
           6       0.93      0.82      0.88        34
           7       0.92      0.81      0.86        27
           8       0.90      0.95      0.92        37
           9       0.94      0.89      0.91        35

    accuracy                           0.83       300
   macro avg       0.84      0.83      0.83       300
weighted avg       0.85      0.83      0.84       300

Test accuracy of the SNN: 0.8333333333333334


In [7]:
model = tf.keras.models.load_model('CIFAR_v19_vgg.keras')
loss, accuracy = model.evaluate(test_data, test_labels_onehot, verbose=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8026 - loss: 0.6756


In [8]:
print(f"Test accuracy of the ANN: {accuracy:.4f}")
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

Test accuracy of the ANN: 0.8026
Test accuracy of the SNN: 0.8333333333333334
